# Predicting NEE on grid

In [ ]:
import sys
import xarray as xr
import numpy as np
from joblib import load
from datacube.utils.dask import start_local_dask

sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/dea_tools')
from classification import predict_xr

In [ ]:
client = start_local_dask(mem_safety_margin='2Gb')
client

### Open model

In [ ]:
model = load('/g/data/os22/chad_tmp/NEE_modelling/results/EBF_NEE_model.joblib').set_params(n_jobs=1)

## Open predictor data

In [ ]:
data = xr.open_dataset('/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_2019.nc')

## Open variable names and index data into correct order

TO DO: make this programmatic

In [ ]:
f = open("/g/data/os22/chad_tmp/NEE_modelling/results/variables.txt", "r")
variables = f.read()
variables = variables.split(",")
print(variables)

In [ ]:
data = data[['tavg','soil_moisture','srad','rain','LAI','vpd','tavg_L1','soil_moisture_L1','srad_L1','rain_L1','LAI_L1','vpd_L1']]

### Predict each time-step seperately

- TO DO: fix timesteps that come back from `predict_xr`

In [ ]:
# data=data.rename({'latitude':'y', 'longitude':'x'})
mask = ~np.isnan(data.LAI.isel(time=1))


In [ ]:
results = []
for i in range(1, len(data)): #start from 1 as forst time-step doesn't have lags
    #print(data.isel(time=i).chunk(dict(x=1000,y=1000)))
    predicted = predict_xr(model,
                        data.isel(time=i).chunk(dict(x=1000,y=1000)),
                        proba=False,
                        clean=True,
                          ).compute()
    
    predicted = predicted.Predictions.where(mask)
    predicted['time'] = data.isel(time=i).time.values
    results.append(predicted)

In [ ]:
nee = xr.concat(results, dim='time')

In [ ]:
nee.plot.imshow(col='time', col_wrap=6);